# 메시지 삭제 방법

그래프의 일반적인 상태 중 하나는 메시지 목록입니다. 일반적으로 이 상태에 메시지를 추가하기만 합니다. 하지만 때때로 메시지를 제거하고 싶을 수 있습니다(상태를 직접 수정하거나 그래프의 일부로서). 이를 위해 `RemoveMessage` 수정자를 사용할 수 있습니다. 이 가이드에서는 그 방법에 대해 설명하겠습니다.

핵심 아이디어는 각 상태 키에 `reducer` 키가 있다는 것입니다. 이 키는 상태의 업데이트를 결합하는 방법을 지정합니다. 기본 `MessagesState`는 메시지 키가 있으며, 해당 키의 리듀서는 이러한 `RemoveMessage` 수정자를 수용합니다. 그 리듀서는 이러한 `RemoveMessage`를 사용하여 키에서 메시지를 삭제합니다.

따라서 그래프 상태에 메시지 목록이 있는 키가 있다고 해서 이 `RemoveMessage` 수정자가 작동할 것이라는 의미는 아닙니다. 이와 함께 작동할 수 있는 `reducer`가 정의되어 있어야 합니다.

**참고**: 많은 모델은 메시지 목록에 대한 특정 규칙을 기대합니다. 예를 들어, 일부는 목록이 `user` 메시지로 시작할 것을 기대하고, 다른 일부는 모든 도구 호출 메시지 뒤에 도구 메시지가 올 것을 기대합니다. **메시지를 삭제할 때 이러한 규칙을 위반하지 않도록 주의해야 합니다.**


## 설정

먼저, 메시지를 사용하는 간단한 그래프를 만들어 보겠습니다. 여기서는 필수 `리듀서`가 포함된 `MessagesState`를 사용하고 있습니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic


다음으로, 우리가 사용할 LLM인 Anthropic의 API 키를 설정해야 합니다.


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 향상시키세요. LangSmith는 추적 데이터를 사용하여 LangGraph로 구축된 LLM 앱을 디버그하고 테스트하며 모니터링할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보세요.
    </p>
</div>


## 에이전트 구축하기
이제 간단한 ReAct 스타일의 에이전트를 구축해 보겠습니다.


In [7]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode

memory = MemorySaver()


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)
model = ChatAnthropic(model_name="claude-3-haiku-20240307")
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the path map - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)


In [8]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. I'm here to help out with any questions or tasks you might have. Please let me know if there's anything I can assist you with.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

You said your name is Bob.


## 수동으로 메시지 삭제하기

먼저, 수동으로 메시지를 삭제하는 방법을 다루겠습니다. 현재 스레드의 상태를 살펴보겠습니다:


In [9]:
messages = app.get_state(config).values["messages"]
messages


[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='db576005-3a60-4b3b-8925-dc602ac1c571'),
 AIMessage(content="It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. I'm here to help out with any questions or tasks you might have. Please let me know if there's anything I can assist you with.", additional_kwargs={}, response_metadata={'id': 'msg_01BKAnYxmoC6bQ9PpCuHk8ZT', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 52}}, id='run-3a60c536-b207-4c56-98f3-03f94d49a9e4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 52, 'total_tokens': 64}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='2088c465-400b-430b-ad80-fad47dc1f2d6'),
 AIMessage(content='You said your name is Bob.', additional_kwargs={}, response_metadata={'id': 'msg_013UWTLTzwZi81vke8mMQ2KP', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'e

우리는 `update_state`를 호출하고 첫 번째 메시지의 ID를 전달할 수 있습니다. 이렇게 하면 해당 메시지가 삭제됩니다.


In [10]:
from langchain_core.messages import RemoveMessage

app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})


{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef75157-f251-6a2a-8005-82a86a6593a0'}}

현재 메시지를 살펴보면 첫 번째 메시지가 삭제되었음을 확인할 수 있습니다.


In [8]:
messages = app.get_state(config).values["messages"]
messages


[AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", response_metadata={'id': 'msg_01XPSAenmSqK8rX2WgPZHfz7', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-1c69af09-adb1-412d-9010-2456e5a555fb-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44}),
 HumanMessage(content="what's my name?", id='f3c71afe-8ce2-4ed0-991e-65021f03b0a5'),
 AIMessage(content='Your name is Bob, as you introduced yourself at the beginning of our conversation.', response_metadata={'id': 'msg_01BPZdwsjuMAbC1YAkqawXaF', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 52, 'output_tokens': 19}}, id='run-b2eb9137-2f4e-446f-95f5-3d5f621a2cf8-0', usage_metadata={'input_tokens': 52, 'output_tokens': 19, 'total_tokens': 71})]

## 프로그래밍 방식으로 메시지 삭제하기

우리는 그래프 내부에서 프로그래밍 방식으로 메시지를 삭제할 수도 있습니다. 여기서는 그래프 실행의 끝에서 3개 이상의 오래된 메시지를 삭제하도록 그래프를 수정할 것입니다.


In [9]:
from langchain_core.messages import RemoveMessage
from langgraph.graph import END


def delete_messages(state):
    messages = state["messages"]
    if len(messages) > 3:
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-3]]}


# We need to modify the logic to call delete_messages rather than end right away
def should_continue(state: MessagesState) -> Literal["action", "delete_messages"]:
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we call our delete_messages function
    if not last_message.tool_calls:
        return "delete_messages"
    # Otherwise if there is, we continue
    return "action"


# Define a new graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# This is our new node we're defining
workflow.add_node(delete_messages)


workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("action", "agent")

# This is the new edge we're adding: after we delete messages, we finish
workflow.add_edge("delete_messages", END)
app = workflow.compile(checkpointer=memory)


이제 이것을 시도해 볼 수 있습니다. 우리는 그래프를 두 번 호출한 다음 상태를 확인할 수 있습니다.


In [10]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "3"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])


[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you.")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), ('human', "what's my name?"), ('ai', 'You said your name is Bob, so that is the name I have for you.')]
[('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), (

현재 상태를 확인하면 세 개의 메시지만 표시되어야 합니다. 이는 우리가 이전 메시지를 삭제했기 때문입니다. 그렇지 않았다면 네 개가 되었을 것입니다!


In [11]:
messages = app.get_state(config).values["messages"]
messages


[AIMessage(content="Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you.", response_metadata={'id': 'msg_01XPEgPPbcnz5BbGWUDWTmzG', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 48}}, id='run-eded3820-b6a9-4d66-9210-03ca41787ce6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 48, 'total_tokens': 60}),
 HumanMessage(content="what's my name?", id='a0ea2097-3280-402b-92e1-67177b807ae8'),
 AIMessage(content='You said your name is Bob, so that is the name I have for you.', response_metadata={'id': 'msg_01JGT62pxhrhN4SykZ57CSjW', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 68, 'output_tokens': 20}}, id='run-ace3519c-81f8-45fe-a777-91f42d48b3a3-0', usage_metadata={'input_tokens': 68, 'output_tokens': 20, 'tot

메시지를 삭제할 때 남아 있는 메시지 목록이 여전히 유효한지 확인해야 합니다. 이 메시지 목록은 **실제로 유효하지 않을 수 있습니다** - 이는 현재 AI 메시지로 시작하고 있기 때문인데, 일부 모델에서는 이것을 허용하지 않기 때문입니다.
